In [8]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'

In [9]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target:cw.targets.SimpleSerial2 = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

INFO: Found ChipWhisperer😍


In [10]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif PLATFORM == "CW308_SAM4S":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None

In [50]:
import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    elif PLATFORM == "CWLITEARM":  
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

def reboot_flush(scope):
    target.flush()
    time.sleep(0.05)
    reset_target(scope)


scope.clock.adc_freq                     changed from 13569877                  to 25543121                 
scope.clock.adc_rate                     changed from 13569877.0                to 25543121.0               
scope.clock.freq_ctr                     changed from 0                         to 2470172                  


In [12]:

def reset_target_speed():
    if scope._is_husky is False:
        if PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
            scope.clock.clkgen_freq = 32E6
            if SS_VER=='SS_VER_2_1':
                target.baud = 230400*32/7.37
            else:
                target.baud = 38400*32/7.37
        elif (PLATFORM == "CWLITEARM") or ("F3" in PLATFORM):
            scope.clock.clkgen_freq = 24E6
            if SS_VER=='SS_VER_2_1':
                target.baud = 230400*24/7.37
            else:
                target.baud = 38400*24/7.37
            target.reset_comms()
            time.sleep(0.1)

In [13]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (15:8-2019-q3-1+b1) 8.3.1 20190703 (release) [gcc-8-branch revision 273027]
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f3/stm32f3_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWLITEARM/stm32f3_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f3 -I.././hal/stm32f3/CMSIS -I.././hal/stm32f3/CMSIS/core -I.././hal/stm32f3/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f

In file included from .././hal/stm32f3/CMSIS/device/stm32f3xx.h:156,
                 from .././hal/stm32f3/stm32f3_hal_lowlevel.h:109,
                 from .././hal/stm32f3/stm32f3_hal.c:3:
.././hal/stm32f3/CMSIS/device/stm32f303xe.h:54:9: note: #pragma message: USING STM32F303xE !!!!!!!!!!!!!!!!!
 #pragma message "USING STM32F303xE !!!!!!!!!!!!!!!!!"
         ^~~~~~~


.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f3/stm32f3_hal_lowlevel.c ...
-e Done!


In file included from .././hal/stm32f3/CMSIS/device/stm32f3xx.h:156,
                 from .././hal/stm32f3/stm32f3_hal_lowlevel.h:109,
                 from .././hal/stm32f3/stm32f3_hal_lowlevel.c:39:
.././hal/stm32f3/CMSIS/device/stm32f303xe.h:54:9: note: #pragma message: USING STM32F303xE !!!!!!!!!!!!!!!!!
 #pragma message "USING STM32F303xE !!!!!!!!!!!!!!!!!"
         ^~~~~~~


.
LINKING:
-en     simpleserial-glitch-CWLITEARM.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-glitch-CWLITEARM.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-glitch-CWLITEARM.elf simpleserial-glitch-CWLITEARM.hex
.
Creating load file for Flash: simpleserial-glitch-CWLITEARM.bin
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature simpleserial-glitch-CWLITEARM.elf simpleserial-glitch-CWLITEARM.bin
.
Creating load file for EEPROM: simpleserial-glitch-CWLITEARM.eep
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex simpleserial-glitch-CWLITEARM.elf simpleserial-glitch-CWLITEARM.eep || exit 0
.
Creating Symbol Table: simpleserial-glitch-CWLITEARM.sym
arm-none-eabi-nm -n simpleserial-glitch-CWLITEARM.elf > simpleserial-glitch-CWLITEARM.sym
.
Creating Extended Listing: simpleserial-glitch-CWLITEARM.lss
arm-none-eabi-objdump -h 

In [14]:
fw_path = "../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

Detected unknown STM32F ID: 0x446
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5671 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5671 bytes


In [15]:
print(target.baud)
print(target.ser._baud)

230400
230400


In [15]:
thing = bytes("rRESET", "latin-1")

for i in range(0, len(thing)):
		print("%02x " % thing[i], end="")

72 52 45 53 45 54 

In [60]:
FRAME_BYTE = 0x00
MAX_SS_LEN = 256
def unstuff_data(buf, len):
	next = buf[0]
	buf[0] = 0x00
	# //len -= 1;
	tmp = next
	while ((next < len) and tmp != 0):
		tmp = buf[next]
		buf[next] = FRAME_BYTE
		next += tmp
	return next, buf


	# uint8_t i = 1;
	# uint8_t last = 0;
	# for (; i < len; i++) {
	# 	if (buf[i] == FRAME_BYTE) {
	# 		buf[last] = i - last;
	# 		last = i;
	# 	}
	# }
	# return 0x00;
def stuff_data(buf, len):
	l = len
	ptr = 0
	last = 0
	for i in range(1, l):
			if (buf[i] == FRAME_BYTE):
					buf[last] = i - last
					last = i
					# target_logger.debug("Stuffing byte {}".format(i))
	return buf

def _calc_crc(buf):
		"""Calculate CRC (0xA6) for buf
		"""
		crc = 0x00
		try:
				for b in buf:
						crc ^= ord(b) if type(b) == str else b
						for _ in range(8):
								if crc & 0x80:
										crc = (crc << 1) ^ 0x4D
										crc &= 0xFF
								else:
										crc <<= 1
										crc &= 0xFF
		except:
				target_logger.error("crc error: {}. Try rebuilding firmware if you only get this error.".format(buf))
		return crc

def format_cmd(c, size, output) -> list[int]:
	data_buf = [0x00 for i in range(MAX_SS_LEN)]
	data_buf[0] = 0x00
	data_buf[1] = c
	data_buf[2] = size
	i = 0
	for i in range(0, size):
		data_buf[i + 3] = output[i]
	i = size
	data_buf[i + 3] = _calc_crc(data_buf[1:])
	data_buf[i + 4] = 0x00
	stuff_data(data_buf, i + 5)
	return data_buf[0:i+5]
 

def unfuck_read(buf, to_char = False):
	for i in range(len(buf)):
		b = buf[i] if type(buf[i]) == int else ord(buf[i])
		
		if (i == 1 or to_char):
			buf[i] = b - 0x40
			buf[i] = chr(buf[i])
		else:
			buf[i] = b & 0x7F
	return buf

next, buf = unstuff_data([0x02,0x74,0x01,0x02,0x75,0x00], 4)

print(next)
print(buf)

error_buf = format_cmd('e',0x01,[1])

for i in range(len(error_buf)):
		b = error_buf[i] if type(error_buf[i]) == int else ord(error_buf[i])
		print(hex(b | 0x80), end=' ')
print()
print(unfuck_read([0x85, 0xA5, 0x81, 0x81, 0xC6, 0x80]))
print(error_buf)

print(unfuck_read([0xB2, 0xB2, 0xA5, 0xB3, 0xA5, 0xB4, 0x80, 0x80, 0x80, 0x8A], True))

5
[0, 116, 0, 0, 117, 0]
0x85 0xe5 0x81 0x81 0xb1 0x80 
[5, 'e', 1, 1, 70, 0]
[5, 'e', 1, 1, 49, 0]
['r', 'r', 'e', 's', 'e', 't', '@', '@', '@', 'J']


In [59]:
reboot_flush(scope)
val = target.read(11, 600)
print(val)
for x in bytes(val, "latin-1"):
    print("%02x " % x , end="")


 
00 

In [17]:

target.simpleserial_write("g", bytearray([]))
val = target.simpleserial_read_witherrors('r', 8, 5000)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']


for x in bytes(val['full_response'], "latin-1"):
    print("%02x " % x , end="")
# print(bytes(val['full_response'], "latin-1"))
print(val)


85 a5 81 81 c6 80 {'valid': False, 'payload': None, 'full_response': '\x85¥\x81\x81Æ\x80', 'rv': None}


In [61]:
if scope._is_husky:
    scope.glitch.enabled = True
    scope.glitch.clk_src = "pll"
    scope.io.glitch_hp = True
    scope.io.glitch_lp = False
else:
    scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True